In [ ]:
|bot_id| bot_Desc| bot_Cate| top_id| top_Desc| top_Cate| tag| bot_sub_catA| bot_sub_catB| bot_sub_catC| bot_sub_catD| top_sub_catA|top_sub_catB| top_sub_catC|top_sub_catD|


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel, MultilayerPerceptronClassifier, MultilayerPerceptronClassificationModel, LinearSVC, LinearSVCModel, LogisticRegression, LogisticRegressionModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import lower, length, size
from pyspark.sql.functions import rand 
from pyspark.sql.functions import lit
from pyspark.sql.functions import split, col
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderModel

In [ ]:
#try with sub cate A
data = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/pre_dl_dataset_textt.csv", inferSchema=True, header=True)
#data = data2.na.drop() c in 

#pre-staging: splitting to sub_cate
data = data.withColumn("bot_sub_catA", split(col("bot_Cate"), ">").getItem(1)).withColumn("bot_sub_catB", split(col("bot_Cate"), ">").getItem(2)).withColumn("bot_sub_catC", split(col("bot_Cate"), ">").getItem(3)).withColumn("bot_sub_catD", split(col("bot_Cate"), ">").getItem(4)).withColumn("top_sub_catA", split(col("top_Cate"), ">").getItem(1)).withColumn("top_sub_catB", split(col("top_Cate"), ">").getItem(2)).withColumn("top_sub_catC", split(col("top_Cate"), ">").getItem(3)).withColumn("top_sub_catD", split(col("top_Cate"), ">").getItem(4)) #.show(2)

#data.printSchema()
#all-in
feature_prep = data.select(lower(data["bot_sub_catA"]).alias("bot_sub_catA"), lower(data["bot_sub_catB"]).alias("bot_sub_catB"),
 lower(data["bot_sub_catC"]).alias("bot_sub_catC"), lower(data["top_sub_catA"]).alias("top_sub_catA"), lower(data["top_sub_catB"]).alias("top_sub_catB"), lower(data["top_sub_catC"]).alias("top_sub_catC"), lower(data["bot_Desc"]).alias("bot_Desc"), lower(data["top_Desc"]).alias("top_Desc"), (data["top_id"]), (data["bot_id"]), "tag")

# Preprocessing and feature engineering

#Cate 2 Cate
stopwordList = ["null", "Null", '', ' '] 
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))

feature_prep = RegexTokenizer(inputCol="bot_sub_catA", outputCol="bot_sub_catAwords", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='bot_sub_catAwords',outputCol='bot_sub_catAstop_words_removed', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="bot_sub_catAstop_words_removed", outputCol="bot_sub_catAtf", numFeatures=50).transform(feature_prep)

feature_prep = RegexTokenizer(inputCol="top_sub_catA", outputCol="top_sub_catAwords2", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='top_sub_catAwords2',outputCol='top_sub_catAstop_words_removed2', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="top_sub_catAstop_words_removed2", outputCol="top_sub_catAtf2", numFeatures=50).transform(feature_prep)
#Desc 2 Desc 
feature_prep = RegexTokenizer(inputCol="bot_Desc", outputCol="words3", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='words3',outputCol='stop_words_removed3', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed3", outputCol="tf3", numFeatures=1000).transform(feature_prep)
feature_prep = IDF(inputCol="tf3", outputCol="tf_idf3").fit(feature_prep).transform(feature_prep)

feature_prep = RegexTokenizer(inputCol="top_Desc", outputCol="words4", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='words4',outputCol='stop_words_removed4', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed4", outputCol="tf4", numFeatures=1000).transform(feature_prep)
feature_prep = IDF(inputCol="tf4", outputCol="tf_idf4").fit(feature_prep).transform(feature_prep)

feature_prep = StringIndexer(inputCol='tag',outputCol='label_indexed').fit(feature_prep).transform(feature_prep)
feature_prep = VectorAssembler(inputCols=["top_id", "bot_id","bot_sub_catAtf","bot_sub_catBtf", "bot_sub_catCtf", "top_sub_catAtf2", "tf_idf3", "tf_idf4"], outputCol="features", handleInvalid="skip").transform(feature_prep)
final_data = feature_prep.select("label_indexed", "features")

# Split data into train and test sets
train_data, test_data = final_data.randomSplit([0.7,0.3])

# Model training

classifier = LogisticRegression(featuresCol="features", labelCol="label_indexed", regParam=0.1, elasticNetParam=1.0, family="multinomial")
classifier = RandomForestClassifier(featuresCol="features", labelCol="label_indexed", numTrees=500, maxDepth=30)
# 200 /25
# 1000/25

model = classifier.fit(train_data)

# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='label_indexed', predictionCol='prediction', metricName='accuracy')
print("Accuracy: {}", evaluator_accuracy.evaluate(predicted_test_data)) 

# Save the model
model.save('hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/ALLIN_classifier_model')
